### Modeling and visualizing dynamics of state COVID cases 

#### Anna Konstorum (konstorum.anna@gmail.com)

In [1]:
# import packages
import datetime
import os
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib.lines import Line2D
%matplotlib inline

In [2]:
print(f'Last updated: {datetime.date.today()}')

Last updated: 2020-04-24


In [4]:
# local directory to save figures if using

In [5]:
# import latest state data
url='https://covidtracking.com/api/states/daily.csv'
state_data=pd.read_csv(url,sep=',',header='infer', parse_dates=[0])

In [6]:
# create exponential function to model 
# will add additional functions shortly

def exp_func(x, a, b, c):
    return a * np.exp(b * x) + c

def quad_func(x,a,b,c):
    return a*x**2 +b*x+c

def lin_func(x,a,b):
    return a*x + b

In [7]:
def SSE_curve(x,y,fxn):
    popt,pop = curve_fit(fxn,x,y)
    y_exp= fxn(x, *popt)
    SSE = sum((y-y_exp)**2)
    return popt,SSE

In [17]:
states=pd.Series(state_data["state"].value_counts())
states=list(states.index)

In [21]:
states_data=[]

#filter for states
for state in states:
    state_df = state_data[state_data["state"]==state]
    state_df=state_df.sort_values(by="date")
    state_df.reset_index(inplace=True,drop=True)
    states_data.append(state_df)

In [ ]:
'''
For each state:
(1) Get min date and max date for full and for each state
(2) Model for min date + 10 to max date
(3) Use x-axis: min an dmax for full
(4) For each state: draw color line corresponding to model 
'''

In [ ]:
# date range of the graph
start_date = min(IL_data['date'])
end_date = max(IL_data['date'])
next_date=end_date+datetime.timedelta(days=1)

print(f'Data from {start_date.strftime("%Y-%m-%d")} to {end_date.strftime("%Y-%m-%d")}')

In [ ]:
# Textual date for labels
tomorrow_label = (end_date + datetime.timedelta(days=1)).strftime('%m/%d')
next2_label = (end_date + datetime.timedelta(days=2)).strftime('%m/%d')
next3_label = (end_date + datetime.timedelta(days=3)).strftime('%m/%d')

# get current date range
date_range=pd.date_range(start=start_date, end=end_date)
date_range_plus=pd.date_range(start=start_date,end=next_date)

# output date range as list
dates = [date.strftime('%m/%d') for date in date_range_plus]

In [ ]:
print(dates[-1])

In [ ]:
# Get fraction of deaths
d_frac_df = pd.DataFrame(index=IL_data.index[13:],columns=['d_frac'])
d_frac_df['date'] = IL_data["date"][13:]
for i in IL_data.index[13:]:
    d = IL_data.iloc[i]['death']
    p = IL_data.iloc[i]['positive']
    d_frac = float("{:.4f}".format(d/p))
    d_frac_df.loc[i,'d_frac']=d_frac*100

In [ ]:
# model fit for positive cases
xdata=np.array(IL_data.index)[-10:]

data_to_model=["positive"]
model_results=pd.DataFrame(index=data_to_model,columns=['a','b','c'])
xdata=np.array(IL_data.index)[-10:]
for item in data_to_model:
    ydata=np.array(IL_data[item])[-10:]
    
    popt_quad,SSE_quad = SSE_curve(xdata,ydata,quad_func)
    popt_lin,SSE_lin = SSE_curve(xdata,ydata,lin_func)
    
    quad_lin_diff=((SSE_lin/SSE_quad)-1)*100
    
    if quad_lin_diff<5:
        no_quad=1
    else:
        no_quad=0   
    
    try:
        popt_exp, SSE_exp = SSE_curve(xdata,ydata,exp_func)
        min_error=min(SSE_exp,SSE_quad,SSE_lin)
    except:
        min_error=min(SSE_quad,SSE_lin)
    
    if (min_error==SSE_quad) and (no_quad==1):
        min_error=SSE_lin
    
    if min_error == SSE_quad:
        model_results.loc[item,['a','b','c']]=popt_quad
        model_results.loc[item,'model']='quad'
    elif min_error == SSE_lin:
        model_results.loc[item,['a','b']]=popt_lin
        model_results.loc[item,'model']='lin'
    elif min_error == SSE_exp:
        model_results.loc[item,['a','b','c']]=popt_exp
        model_results.loc[item,'model']='exp'    
        

In [ ]:
model_results

In [ ]:
# Include all dates, even those with missing data
xdata_full=np.array(range(len(dates)))
xdata_pred=np.array(range(len(dates)))[-11:]
xdata_dfrac = np.array(range(len(dates)))[13:-1]

In [ ]:
#func use
def function_use(county,model_results):
    if model_results.loc[county]["model"]=='exp':
        func_use=exp_func
        opt_model = 'Exponential'
    elif model_results.loc[county]["model"]=='quad':
        func_use=quad_func
        opt_model = 'Quadratic (up)'
        if model_results.loc[county]['a']<0:
            opt_model = "Quadratic (down)"
    else:
        func_use=lin_func
        opt_model = 'Linear'
    return func_use, opt_model

### Illinois model and dynamics

In [ ]:
next_day = xdata_pred[-1]
next2_day = next_day+1; next3_day = next_day+2
sub="[1] Schools close; [2] Stay-at-home order"

In [ ]:
popt_state = model_results.loc["positive"][0:3]

func_use, opt_model = function_use("positive",model_results)

if opt_model=="Linear":
    next_day_cases = func_use(next_day, popt_state[0], popt_state[1])
    next2_day_cases = func_use(next2_day, popt_state[0], popt_state[1])
    next3_day_cases = func_use(next3_day, popt_state[0], popt_state[1])
    
else:
    next_day_cases = func_use(next_day, popt_state[0], popt_state[1], popt_state[2])
    next2_day_cases = func_use(next2_day, popt_state[0], popt_state[1], popt_state[2])
    next3_day_cases = func_use(next3_day, popt_state[0], popt_state[1], popt_state[2])


loc = "Illinois"
model = "Best fit model: {}"
pred_text="Prediction for total cases:"
next_pred='\t{}: {}';

In [ ]:
legend_elements =  [Line2D([0], [0], marker='+', color='red', label=loc, markersize=15),
                    Line2D([0], [0], marker='', color='black',linestyle='', label=model.format(opt_model), markersize=15),
                Line2D([1], [0], marker='', color='black',linestyle='', label=pred_text, markersize=1),
                   Line2D([1], [0], marker='', color='black',linestyle='', label=next_pred.format(tomorrow_label,int(next_day_cases)), markersize=1),
                   Line2D([0], [0], marker='', color='black',linestyle='', label=next_pred.format(next2_label,int(next2_day_cases)), markersize=15),
                   Line2D([0], [0], marker='', color='black',linestyle='', label=next_pred.format(next3_label,int(next3_day_cases)), markersize=15)]

if opt_model=='Linear':
    popt_state=popt_state[0:2]

plt.figure(figsize=(15,10))

plt.plot(xdata_pred, func_use(xdata_pred, *popt_state), color='grey',linestyle='--');
plt.scatter(IL_data.index,IL_data['positive'],color='red',marker='+');

plt.vlines(x=12, ymin=0, ymax=500,color='dimgrey', linewidth=0.5,linestyle='-.');
plt.text(12.4,490,"[1]",rotation=0,color='black',fontsize = 13,verticalalignment='center',horizontalalignment='center');

plt.vlines(x=16, ymin=0, ymax=1000,color='dimgrey', linewidth=0.5,linestyle='-.');
plt.text(16.4,900,"[2]",rotation=0,color='black',fontsize = 13,verticalalignment='center',horizontalalignment='center');

plt.ylabel('# COVID cases',fontsize=15);
plt.xticks(xdata_full, dates, rotation=45,fontsize=12)  
plt.yticks(fontsize=12)
plt.xlabel('Dates (2020)',fontsize=15)
plt.legend(handles=legend_elements,fontsize=15,loc='upper left')
plt.suptitle("COVID dynamics for Illinois",fontsize=20);
plt.title(sub, fontsize=13);
plt.savefig("IL_update.png")

In [ ]:
legend_elements =  [Line2D([0], [0], marker='+', color='slateblue', label="Crude CFR (Case Fatality Rate)", markersize=15)]

plt.figure(figsize=(15,10))

plt.plot(xdata_dfrac,d_frac_df["d_frac"],'o-',color='slateblue');
                               
plt.vlines(x=12, ymin=0, ymax=2,color='dimgrey', linewidth=0.5,linestyle='-.');
plt.text(12.4,2,"[1]",rotation=0,color='black',fontsize = 13,verticalalignment='center',horizontalalignment='center');

plt.vlines(x=16, ymin=0, ymax=2,color='dimgrey', linewidth=0.5,linestyle='-.');
plt.text(16.4,2,"[2]",rotation=0,color='black',fontsize = 13,verticalalignment='center',horizontalalignment='center');                              

plt.ylabel('Crude CFR (%)',fontsize=15);
plt.xticks(xdata_full, dates, rotation=45,fontsize=12)  
plt.yticks(fontsize=12)
plt.xlabel('Dates (2020)',fontsize=15)
plt.legend(handles=legend_elements,fontsize=15,loc='upper left')
plt.suptitle("Crude CFR for Illinois",fontsize=20);
plt.title(sub, fontsize=13);
plt.savefig("IL_update_p.png")

In [ ]:
##import latest county data
url='https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
county_data=pd.read_csv(url,sep=',',header='infer', parse_dates=[0])

In [ ]:
state = county_data[county_data["state"]=='Illinois']
state['date_v2']=state['date'].dt.strftime('%m/%d')

In [ ]:
state.tail()

In [ ]:
state_f = state["date"]>="2020-03-04"
state_f = state[state_f]
counties=["McHenry","Lake","Cook"]

In [ ]:
start_date=min(state_f["date"])
end_date=max(state_f["date"])
next_date=end_date+datetime.timedelta(days=1)
date_range=pd.date_range(start=start_date, end=end_date)
date_range_plus=pd.date_range(start=start_date,end=next_date)
dates = [date.strftime('%m-%d-%y') for date in date_range]
dates_plus = [date.strftime('%m-%d-%y') for date in date_range_plus]

tomorrow_label = (end_date + datetime.timedelta(days=1)).strftime('%m/%d')
next2_label = (end_date + datetime.timedelta(days=2)).strftime('%m/%d')
next3_label = (end_date + datetime.timedelta(days=3)).strftime('%m/%d')

In [ ]:
county_d = pd.DataFrame(index=dates, columns=counties)
county_d.fillna(0,inplace=True)
county_c = pd.DataFrame(index=dates, columns=counties)
county_c.fillna(0,inplace=True)

for county in counties:
    county_spec = state_f[state_f["county"]==county]
    county_spec.index=county_spec['date']
    county_dates = [date.strftime('%m-%d-%y') for date in county_spec.index]
    for county_date in county_dates:
        county_c.loc[county_date,county]=county_spec.loc[county_date]["cases"]
        county_d.loc[county_date,county]=county_spec.loc[county_date]["deaths"]
        
county_c.reset_index(inplace=True,drop=False)
county_c.rename(columns={"index":"date"},inplace=True)

county_d.reset_index(inplace=True,drop=False)
county_d.rename(columns={"index":"date"},inplace=True)

In [ ]:
# model fit for positive cases
xdata=np.array(county_c.index)[-10:]

select_counties=["Lake","Cook","McHenry"]
model_results_county=pd.DataFrame(index=select_counties,columns=['a','b','c'])
xdata=np.array(county_c.index)[-10:]
for item in select_counties:
    ydata=np.array(county_c[item])[-10:]
    popt_quad,SSE_quad = SSE_curve(xdata,ydata,quad_func)
    popt_lin,SSE_lin = SSE_curve(xdata,ydata,lin_func)
    
    quad_lin_diff=((SSE_lin/SSE_quad)-1)*100
    
    if quad_lin_diff<5:
        no_quad=1
    else:
        no_quad=0   
    
    try:
        popt_exp, SSE_exp = SSE_curve(xdata,ydata,exp_func)
        min_error=min(SSE_exp,SSE_quad,SSE_lin)
    except:
        min_error=min(SSE_quad,SSE_lin)
    
    if (min_error==SSE_quad) and (no_quad==1):
        min_error=SSE_lin
    
    if min_error == SSE_quad:
        model_results_county.loc[item,['a','b','c']]=popt_quad
        model_results_county.loc[item,'model']='quad'
    elif min_error == SSE_lin:
        model_results_county.loc[item,['a','b']]=popt_lin
        model_results_county.loc[item,'model']='lin'
    elif min_error == SSE_exp:
        model_results_county.loc[item,['a','b','c']]=popt_exp
        model_results_county.loc[item,'model']='exp'    

In [ ]:
model_results_county

In [ ]:
# Include all dates, even those with missing data
xdata_full=np.array(range(len(dates_plus)))
xdata_pred=np.array(range(len(dates_plus)))[-11:]

In [ ]:
next_day = xdata_pred[-1]
next2_day = next_day+1; next3_day = next_day+2
sub="[1] Schools close; [2] Stay-at-home order"

In [ ]:
# Cook County

In [ ]:
poptC = model_results_county.loc["Cook"][0:3]

func_useC, opt_modelC = function_use("Cook",model_results_county)

if opt_modelC=="Linear":
    next_day_casesC = func_useC(next_day, poptC[0], poptC[1])
    next2_day_casesC = func_useC(next2_day, poptC[0], poptC[1])
    next3_day_casesC = func_useC(next3_day, poptC[0], poptC[1])

else:
    next_day_casesC = func_useC(next_day, poptC[0], poptC[1], poptC[2])
    next2_day_casesC = func_useC(next2_day, poptC[0], poptC[1], poptC[2])
    next3_day_casesC = func_useC(next3_day, poptC[0], poptC[1], poptC[2])

locC = "Cook County"
model = "Best fit model: {}"
pred_text="Prediction for total cases:"
next_pred='\t{}: {}';

In [ ]:
# Create legend
legend_elements =  [Line2D([0], [0], marker='+', color='maroon', label=locC, markersize=15),
                    Line2D([0], [0], marker='', color='black',linestyle='', label=model.format(opt_modelC), markersize=15),
                    Line2D([1], [0], marker='', color='black',linestyle='', label=pred_text, markersize=1),
                    Line2D([1], [0], marker='', color='black',linestyle='', label=next_pred.format(tomorrow_label,int(next_day_casesC)), markersize=1),
                    Line2D([0], [0], marker='', color='black',linestyle='', label=next_pred.format(next2_label,int(next2_day_casesC)), markersize=15),
                    Line2D([0], [0], marker='', color='black',linestyle='', label=next_pred.format(next3_label,int(next3_day_casesC)), markersize=15)]
                
if opt_modelC=='Linear':
    poptC=poptC[0:2]
    
plt.figure(figsize=(15,10));

plt.plot(xdata_pred, func_useC(xdata_pred, *poptC), color='grey',linestyle='--')
plt.scatter(county_c.index,county_c['Cook'],color='maroon',marker='+')

plt.vlines(x=12, ymin=0, ymax=250,color='dimgrey', linewidth=0.5,linestyle='-.');
plt.text(12.4,245,"[1]",rotation=0,color='black',fontsize = 13,verticalalignment='center',horizontalalignment='center');

plt.vlines(x=16, ymin=0, ymax=900,color='dimgrey', linewidth=0.5,linestyle='-.');
plt.text(16.4,895,"[2]",rotation=0,color='black',fontsize = 13,verticalalignment='center',horizontalalignment='center');

plt.ylabel('# COVID cases',fontsize=15);
plt.xticks(xdata_full, dates_plus, rotation=45,fontsize=12)  
plt.yticks(fontsize=12)
plt.xlabel('Dates (2020)',fontsize=15)
plt.legend(handles=legend_elements,fontsize=15,loc='upper left')
plt.suptitle("COVID dynamics for Cook county in Illinois",fontsize=20); 
plt.title(sub, fontsize=13);
plt.savefig("IL_update_Cook.png")

In [ ]:
# Lake AND McHenry

In [ ]:
poptL = model_results_county.loc["Lake"][0:3]
poptM = model_results_county.loc["McHenry"][0:3]

func_useL, opt_modelL = function_use("Lake",model_results_county)
func_useC, opt_modelM = function_use("McHenry",model_results_county)

if opt_modelL=="Linear":
    next_day_casesL = func_useL(next_day, poptL[0], poptL[1])
    next2_day_casesL = func_useL(next2_day, poptL[0], poptL[1])
    next3_day_casesL = func_useL(next3_day, poptL[0], poptL[1])

else:
    next_day_casesL = func_useL(next_day, poptL[0], poptL[1], poptL[2])
    next2_day_casesL = func_useL(next2_day, poptL[0], poptL[1], poptL[2])
    next3_day_casesL = func_useL(next3_day, poptL[0], poptL[1], poptL[2])

if opt_modelM=="Linear":
    next_day_casesM = func_useC(next_day, poptM[0], poptM[1])
    next2_day_casesM = func_useC(next2_day, poptM[0], poptM[1])
    next3_day_casesM = func_useC(next3_day, poptM[0], poptM[1])

else:
    next_day_casesM = func_useC(next_day, poptM[0], poptM[1],poptM[2])
    next2_day_casesM = func_useC(next2_day, poptM[0], poptM[1],poptM[2])
    next3_day_casesM = func_useC(next3_day, poptM[0], poptM[1],poptM[2])

locL = "Lake County"
locM = "McHenry County"
model = "Best fit model: {}"
pred_text="Prediction for total cases:"
next_pred='\t{}: {}';

In [ ]:
# Create legend
legend_elements =  [Line2D([0], [0], marker='+', color='steelblue', label=locL, markersize=15),
                    Line2D([0], [0], marker='', color='black',linestyle='', label=model.format(opt_modelL), markersize=15),
                    Line2D([1], [0], marker='', color='black',linestyle='', label=pred_text, markersize=1),
                    Line2D([1], [0], marker='', color='black',linestyle='', label=next_pred.format(tomorrow_label,int(next_day_casesL)), markersize=1),
                    Line2D([0], [0], marker='', color='black',linestyle='', label=next_pred.format(next2_label,int(next2_day_casesL)), markersize=15),
                    Line2D([0], [0], marker='', color='black',linestyle='', label=next_pred.format(next3_label,int(next3_day_casesL)), markersize=15),
                    Line2D([0], [0], marker='+', color='orange', label=locM, markersize=15),
                    Line2D([0], [0], marker='', color='black',linestyle='', label=model.format(opt_modelM), markersize=15),
                    Line2D([1], [0], marker='', color='black',linestyle='', label=pred_text, markersize=1),
                    Line2D([1], [0], marker='', color='black',linestyle='', label=next_pred.format(tomorrow_label,int(next_day_casesM)), markersize=1),
                    Line2D([0], [0], marker='', color='black',linestyle='', label=next_pred.format(next2_label,int(next2_day_casesM)), markersize=15),
                    Line2D([0], [0], marker='', color='black',linestyle='', label=next_pred.format(next3_label,int(next3_day_casesM)), markersize=15)]

if opt_modelL=='Linear':
    poptL=poptL[0:2]

if opt_modelM=='Linear':
    poptM=poptM[0:2]
    
plt.figure(figsize=(15,10));

plt.plot(xdata_pred, func_useL(xdata_pred, *poptL), color='grey',linestyle='--')
plt.scatter(county_c.index,county_c['Lake'],color='steelblue',marker='+')

plt.plot(xdata_pred, func_useC(xdata_pred, *poptM), color='grey',linestyle='--')
plt.scatter(county_c.index,county_c['McHenry'],color='orange',marker='+')

plt.vlines(x=12, ymin=0, ymax=50,color='dimgrey', linewidth=0.5,linestyle='-.');
plt.text(12.4,45,"[1]",rotation=0,color='black',fontsize = 13,verticalalignment='center',horizontalalignment='center');

plt.vlines(x=16, ymin=0, ymax=100,color='dimgrey', linewidth=0.5,linestyle='-.');
plt.text(16.4,95,"[2]",rotation=0,color='black',fontsize = 13,verticalalignment='center',horizontalalignment='center');

plt.ylabel('# COVID cases',fontsize=15);
plt.xticks(xdata_full, dates_plus, rotation=45,fontsize=12)  
plt.yticks(fontsize=12)
plt.xlabel('Dates (2020)',fontsize=15)
plt.legend(handles=legend_elements,fontsize=15,loc='upper left')
plt.suptitle("COVID dynamics for two Illinois counties: Lake and McHenry",fontsize=20); 
plt.title(sub, fontsize=13);
plt.savefig("IL_update_Lake_McHenry.png")

In [ ]:
# Get fraction of deaths
d_frac_dfC = pd.DataFrame(index=county_c.index,columns=['d_frac'])
d_frac_dfC['date'] = county_c["date"]
for i in county_c.index:
    d = county_d.iloc[i]['Cook']
    p = county_c.iloc[i]['Cook']
    if p!=0:
        d_frac = float("{:.4f}".format(d/p))
        d_frac_dfC.loc[i,'d_frac']=d_frac*100

# Get fraction of deaths
d_frac_dfL = pd.DataFrame(index=county_c.index,columns=['d_frac'])
d_frac_dfL['date'] = county_c["date"]
for i in county_c.index:
    d = county_d.iloc[i]['Lake']
    p = county_c.iloc[i]['Lake']
    if p!=0:
        d_frac = float("{:.4f}".format(d/p))
        d_frac_dfL.loc[i,'d_frac']=d_frac*100
        
# Get fraction of deaths
d_frac_dfM = pd.DataFrame(index=county_c.index,columns=['d_frac'])
d_frac_dfM['date'] = county_c["date"]
for i in county_c.index:
    d = county_d.iloc[i]['McHenry']
    p = county_c.iloc[i]['McHenry']
    if p!=0:
        d_frac = float("{:.4f}".format(d/p))
        d_frac_dfM.loc[i,'d_frac']=d_frac*100

In [ ]:
legend_elements =  [Line2D([0], [0], marker='+', color='seagreen', label="Lake County", markersize=15),
                   Line2D([0], [0], marker='+', color='mediumorchid', label="McHenry County", markersize=15),
                   Line2D([0], [0], marker='+', color='mediumblue', label="Cook County", markersize=15)]

plt.figure(figsize=(15,10))

plt.plot(d_frac_dfL.index,d_frac_dfL["d_frac"],'o-',color='seagreen');
plt.plot(d_frac_dfM.index,d_frac_dfM["d_frac"],'o-',color='mediumorchid');
plt.plot(d_frac_dfC.index,d_frac_dfC["d_frac"],'o-',color='mediumblue');
                               
plt.vlines(x=12, ymin=0, ymax=1.5,color='dimgrey', linewidth=0.5,linestyle='-.');
plt.text(12.4,1.4,"[1]",rotation=0,color='black',fontsize = 13,verticalalignment='center',horizontalalignment='center');

plt.vlines(x=16, ymin=0, ymax=1.5,color='dimgrey', linewidth=0.5,linestyle='-.');
plt.text(16.4,1.4,"[2]",rotation=0,color='black',fontsize = 13,verticalalignment='center',horizontalalignment='center');                              

plt.ylabel('Crude CFR (%)',fontsize=15);
plt.xticks(xdata_full, dates, rotation=45,fontsize=12)  
plt.yticks(fontsize=12)
plt.xlabel('Dates (2020)',fontsize=15)
plt.legend(handles=legend_elements,fontsize=15,loc='upper left')
plt.suptitle("Crude CFR (Case Fatality Rate) for Cook, Lake, and McHenry County",fontsize=20);
plt.title(sub, fontsize=13);
plt.savefig("IL_update_county_p.png")